## Python in your GPU 1 : Compute PI

In [1]:
import numpy as np
import numba
from numba import cuda

## Simple python 
[ https://en.wikipedia.org/wiki/Monte_Carlo_integration ]

In [2]:
def calculate_pi(trials):
    count = 0
    for i in range(trials):
        x = np.random.uniform(0, 1)
        y = np.random.uniform(0, 1)
        if x**2 + y**2 <= 1:
            count += 1
    return  count / trials * 4

In [3]:
%time calculate_pi(1_000_000)

CPU times: user 2.48 s, sys: 99 µs, total: 2.48 s
Wall time: 2.48 s


3.140332

## Numpy

In [4]:
def calculate_pi_numpy(trials):
    x = np.random.uniform(0, 1, trials)    
    y = np.random.uniform(0, 1, trials)
    return np.mean(x**2 + y**2 <= 1) * 4

In [5]:
%time calculate_pi_numpy(100_000_000)

CPU times: user 1.53 s, sys: 335 ms, total: 1.87 s
Wall time: 1.87 s


3.14162884

In [6]:
def calculate_pi_numpy2(trials):
    return np.mean( np.random.uniform(0, 1, trials)**2 + np.random.uniform(0, 1, trials)**2 <= 1 ) * 4

In [7]:
%time calculate_pi_numpy2(100_000_000)

CPU times: user 1.51 s, sys: 192 ms, total: 1.7 s
Wall time: 1.7 s


3.14116828

## Numba JIT
[ https://numba.pydata.org/ ]

In [8]:
@numba.njit()
def calculate_pi_numba(trials):
    count = 0
    for i in range(trials):
        x = np.random.uniform(0, 1)
        y = np.random.uniform(0, 1)
        if x**2 + y**2 <= 1:
            count += 1
    return  count / trials * 4

In [9]:
%time calculate_pi_numba(100_000_000)

CPU times: user 905 ms, sys: 27 ms, total: 932 ms
Wall time: 952 ms


3.1418004

In [10]:
%time calculate_pi_numba(100_000_000)

CPU times: user 672 ms, sys: 791 µs, total: 673 ms
Wall time: 661 ms


3.14182128

## Numba Cuda 
[ https://numba.readthedocs.io/en/stable/cuda/index.html ]

In [11]:
from numba.cuda.random import create_xoroshiro128p_states
from numba.cuda.random import xoroshiro128p_uniform_float32

In [12]:
@cuda.jit()
def calculate_pi_numba_cuda_kernel(trials_per_thread, count_array, rng_states):
    my_index = cuda.grid(1)
    my_count = 0
    
    for i in range(trials_per_thread):
        x = xoroshiro128p_uniform_float32(rng_states, my_index)
        y = xoroshiro128p_uniform_float32(rng_states, my_index)
        if x**2 + y**2 <= 1:
            my_count += 1
    count_array[my_index] = my_count
             

In [13]:
def calculate_pi_numba_cuda(trials_per_thread, threads_per_threadblock, num_threadblocks, seed):
    num_threads = threads_per_threadblock * num_threadblocks
    print(f'Computing PI using {trials_per_thread * num_threads:.3e} trials')
    
    # Setup for doing computations on GPU
    rng_states = create_xoroshiro128p_states(num_threads, seed)
    count_array = cuda.to_device(np.zeros(num_threads, dtype=np.int32))
    
    calculate_pi_numba_cuda_kernel[num_threadblocks, threads_per_threadblock](trials_per_thread, count_array, rng_states)
    #calculate_pi_numba_cuda_kernel.forall(num_threads)(trials_per_thread, count_array, rng_states)
    
    # Retrieve and report results
    pi_table = count_array.copy_to_host()/trials_per_thread*4
    print(pi_table)
    return np.mean(pi_table), np.std(pi_table)/len(pi_table)**.5

In [14]:
%time calculate_pi_numba_cuda(100_000, 1024, 1000, seed=12) 

Computing PI using 1.024e+11 trials
[3.1398  3.14116 3.14064 ... 3.13972 3.13756 3.13308]
CPU times: user 3.29 s, sys: 112 ms, total: 3.4 s
Wall time: 3.49 s


(3.141596157343749, 5.131469625478166e-06)

In [15]:
%time calculate_pi_numba_cuda(100_000, 1024, 1000, seed=12) 

Computing PI using 1.024e+11 trials
[3.1398  3.14116 3.14064 ... 3.13972 3.13756 3.13308]
CPU times: user 2.81 s, sys: 34 µs, total: 2.81 s
Wall time: 2.81 s


(3.141596157343749, 5.131469625478166e-06)

In [16]:
np.pi

3.141592653589793

## Perfomance

In [17]:
Trials/s            Ratio   Version

1e6/3.36 =  3.98e5  0.0057  Python

1e8/1.45 =  6.90e7  1       Numpy

1e8/0.701 = 1.43e8  2.1     Numba JIT

1e11/4.54 = 2.20e10 319     Numba Cuda

SyntaxError: invalid syntax (892190816.py, line 1)